# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

In [26]:
import pandas as pd
import numpy as np
from scipy import stats
import math

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### Q1: What test is appropriate for this problem? Does CLT apply?

In [9]:
data.groupby(['race', 'call'])[['id']].count()

id
race call      
b    0.0   2278
     1.0    157
w    0.0   2200
     1.0    235

Large number of cases in each group, can apply the CLT. This problem involves proportions and I will use a two proportion z-test to compare the callback rates for the black- and white-sounding names.

### Q2: What are the null and alternative hypotheses?

Null: The proportion of callbacks for black-sounding and white-sounding names is the same.


Alternative: The proportion of callbacks for black- and white-sounding names is different.

### Q3: Compute margin of error, confidence interval, and p-value.


Margin of Error = critical value x Standard Error of statistic

critical value (z_0.975 or z_0.025) = 1.96

Calculate proportions and standard error

In [19]:
#number of applications for black- and white-sounding names
nb = sum(data.race == 'b')
nw = sum(data.race == 'w')

#number of callbacks for black- and white-sounding names
cb = sum(data[data.race=='b'].call)
cw = sum(data[data.race == 'w'].call)

#proportion of callbacks 
pb = cb / nb
pw = cw / nw

print('proportion of black-sounding callbacks:', pb, ' proportion of white-sounding callbacks:', pw)

proportion of black-sounding callbacks: 0.064476386037  proportion of white-sounding callbacks: 0.0965092402464


In [27]:
#calculate standard error
se = math.sqrt((pb * (1 - pb)) / nb + (pw * (1 - pw)) / nw)
print('The standard error of the difference of population proportions is ', se)

The sranadard error of the difference of population proportions is  0.0077833705866767544


In [30]:
#calculate margin of error 
me = 1.96 * se
print('The margin of error (95% confidence interval) of the difference of population proportions is +/- ', me)


The margin of error (95% confidence interval) of the difference of population proportions is +/-  0.015255406349886438


In [32]:
# difference between proportions
pdif = pw - pb
print('The difference between the population proportions is ', pdif)


The difference between the population proportions is  0.0320328542094


In [33]:
#range of the 95% confidence interval for the difference in population proportions

print('The 95% confidence interval for the difference in poplation proportions is ', pdif - me, ' to ', pdif + me)

The 95% confidence interval for the difference in poplation proportions is  0.0167774478596  to  0.0472882605593


In [34]:
#compute z statistic to get p value
z = pdif / se
pval =  (1 - stats.norm.cdf(z)) * 2

print('The z-test statistic is ', z, ', which produces a p-value of ', pval, 'for the difference in population proportions.')

The z-test statistic is  4.11555043573 , which produces a p-value of  3.86256520752e-05 for the difference in population proportions.


### Q4: Data story

Our analysis indicates that we are 95% confident that the difference in the proportion of callbacks between applications with white-sounding and black-sounding names is between 1.68% and 4.73%.  This difference seems small, but considering the values of the proportion of callbacks are temselves small (6.45% and 9.56% for black-sounding and white-sounding names, respecetively), this difference is actually a large effect.  

The p-value for the two-proportion z-test is 3.86e-5, which is below the critical value of 0.05 and provides strong evidence to reject the null hypothesis.  The data suggest that there is bias agianst applicants with black-sounding names getting a callback after submitting their resumes.  

### Q5: Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

This analysis does not indicate that the sound of the name is the most important factor in getting a callback.  Other factors that were not investigated could be that the applicants with white-sounding names could have more years of experience or that the applicants with black-sounding names could be applying for jobs in sectors that have lower callback rates (Simpson's paradox).  I did some data exploration below that suggests that these factors are likley not producing the difference in callback rates, but there could still be other unexplored factors that are important.

In [45]:
data.groupby(['race', 'call'])[['yearsexp']].mean()

yearsexp
race call          
b    0.0   7.745391
     1.0   9.050955
w    0.0   7.757273
     1.0   8.782979

In [43]:
data.groupby(['race'])[['manuf', 'transcom', 'bankreal', 'trade', 'busservice', 'othservice']].sum()

,manuf,transcom,bankreal,trade,busservice,othservice
race,,,,,,
b,202.0,74.0,207.0,521.0,652.0,377.0
w,202.0,74.0,207.0,521.0,652.0,377.0
